In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import geopandas as gpd
import statsmodels.formula.api as smf
from plotnine import *
pd.options.mode.chained_assignment = None
energy = pd.read_csv("https://data.cityofnewyork.us/api/views/5zyy-y8am/rows.csv?date=20231212&accessType=DOWNLOAD")
covered_buildings = pd.read_csv("../data/covered_buildings.csv")

energy['BBL'] = energy['NYC Borough, Block and Lot (BBL)'].str.replace('-', '')
energy = energy[energy['BBL'].isin(covered_buildings['10 Digit BBL'].astype(str))]

/var/folders/ct/9lwc0fbn26v_v6pz1pfj1dxc0000gq/T/ipykernel_10454/3516254262.py:10: DtypeWarning: Columns (10,78) have mixed types. Specify dtype option on import or set low_memory=False.


In [80]:
energy["Multifamily Housing - Number of Laundry Hookups in All Units"]
energy["Multifamily Housing - Number of Laundry Hookups in Common Area(s)"]

0        Not Available
1        Not Available
2        Not Available
3                    4
4        Not Available
             ...      
30480    Not Available
30481    Not Available
30482    Not Available
30483    Not Available
30484    Not Available
Name: Multifamily Housing - Number of Laundry Hookups in Common Area(s), Length: 27636, dtype: object

In [93]:
housing = energy[energy['Primary Property Type - Portfolio Manager-Calculated'] == "Multifamily Housing"]
housing['total_units'] = pd.to_numeric(housing['Multifamily Housing - Total Number of Residential Living Units'], errors='coerce')
housing['co2_intensity'] = pd.to_numeric(housing['Total (Location-Based) GHG Emissions Intensity (kgCO2e/ft²)'], errors='coerce')
housing['EUI'] = pd.to_numeric(housing['Site EUI (kBtu/ft²)'], errors='coerce')
housing['housing_fa'] = pd.to_numeric(housing['Multifamily Housing - Gross Floor Area (ft²)'], errors='coerce')
housing['total_fa'] = pd.to_numeric(housing['Property GFA - Self-Reported (ft²)'], errors='coerce')
housing["total_co2"] = pd.to_numeric(housing['Total (Location-Based) GHG Emissions (Metric Tons CO2e)'], errors='coerce') 
housing["bedrooms"] = pd.to_numeric(housing["Multifamily Housing - Number of Bedrooms"], errors='coerce')
housing["direct_emissions"] =  pd.to_numeric(housing["Direct GHG Emissions (Metric Tons CO2e)"], errors='coerce')
housing["indirect_emissions"] = pd.to_numeric(housing["Indirect (Location-Based) GHG Emissions (Metric Tons CO2e)"], errors='coerce')
housing["kwh"] = pd.to_numeric(housing["Electricity Use - Grid Purchase (kWh)"], errors='coerce')
housing["gallons_water"] = pd.to_numeric(housing["Water Use (All Water Sources) (kgal)"], errors='coerce') * 1000
housing["unit_laundry_hookups"] = pd.to_numeric(energy["Multifamily Housing - Number of Laundry Hookups in All Units"], errors="coerce")
housing["common_laundry_hookups"] = pd.to_numeric(energy["Multifamily Housing - Number of Laundry Hookups in Common Area(s)"], errors="coerce")


In [94]:
housing[["unit_laundry_hookups", "common_laundry_hookups"]].sample(20)


,unit_laundry_hookups,common_laundry_hookups
20330,0.0,3.0
23680,NaN,NaN
6712,0.0,0.0
2500,NaN,NaN
19875,NaN,NaN
10813,NaN,NaN
5260,0.0,0.0
8945,34.0,0.0
13315,0.0,0.0
17838,0.0,0.0


In [96]:
housing_trim = housing[['BBL', 'Property Name', 'Property ID', 'total_units', 'EUI', 'bedrooms', 
                        'housing_fa', 'total_fa', 'co2_intensity', "total_co2", "kwh", 'gallons_water', 'Latitude', 
                        'Longitude', "Year Built", "Occupancy", "unit_laundry_hookups", "common_laundry_hookups",
                        "direct_emissions", "indirect_emissions"]].rename(columns={'Property Name': 'name', 'Census Tract' : "GEOID", "Year Built": "year_built"}).query("total_units > 0")

housing_trim['total_E'] = housing_trim['EUI'] * housing_trim['total_fa']
housing_trim["kwh_per_sqft"] = housing_trim['kwh'] / housing_trim['total_fa']
housing_trim['avg_apt_size'] = housing_trim['housing_fa'] / housing_trim['total_units']
housing_trim['E_per_apt'] = housing_trim['total_E'] / housing_trim['total_units']
housing_trim['co2_tons_sq'] = housing_trim['co2_intensity'] / 1000
housing_trim['co2_per_apt'] = (housing_trim['co2_tons_sq'] * housing_trim['total_fa']) / housing_trim['total_units']
housing_trim['co2_per_bedroom'] = (housing_trim['co2_tons_sq'] * housing_trim['total_fa']) / housing_trim['bedrooms']
housing_trim['exceeds_2024_limit'] = housing_trim['co2_tons_sq'] > 0.00675
housing_trim['exceeds_2030_limit'] = housing_trim['co2_tons_sq'] > 0.00407
housing_trim['decade_built'] = (np.floor(housing_trim["year_built"]/10)*10).astype(int)
housing_trim['bedrooms_per_apt'] =  housing_trim["bedrooms"] / housing_trim["total_units"]
housing_trim['sqft_per_bedroom'] = housing_trim['housing_fa'] / housing_trim["bedrooms"]
housing_trim["direct_emission_pct"] = housing_trim["direct_emissions"] / housing_trim["total_co2"]
housing_trim["water_per_sqft"] = housing_trim["gallons_water"] / housing_trim['total_fa']
housing_trim["water_per_apt"] = housing_trim["gallons_water"] / housing_trim['total_units']
housing_trim["laundry_per_apt"] = housing_trim["unit_laundry_hookups"] / housing_trim['total_units']

In [97]:
housing_trim["laundry_cat"] = np.select(
    [
        housing_trim["common_laundry_hookups"].isna(),
        housing_trim["laundry_per_apt"] > .5,
        housing_trim["common_laundry_hookups"] > 0,
    ],
    [
        "Unkownn",
        "In-unit",
        "In-building"
    ],
    default = "None"

)

In [103]:
housing_trim["efficiency_quintile"] = pd.qcut(housing_trim["co2_tons_sq"], 5, labels=[1,2,3,4,5])
housing_trim["unit_co2_quintile"] = pd.qcut(housing_trim["co2_per_apt"], 5, labels=[1,2,3,4,5])
housing_trim["bedroom_co2_quintile"] = pd.qcut(housing_trim["co2_per_bedroom"], 5, labels=[1,2,3,4,5])
housing_trim["electricty_sqft_quintile"] = pd.qcut(housing_trim["kwh_per_sqft"], 5, labels=[1,2,3,4,5])

/Users/canyonfoot/Documents/python_proj/Building-Emissions/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract


In [104]:
# 1 row has missing value.. 11 have inf
housing_trim = housing_trim.dropna(subset=["sqft_per_bedroom"])

In [105]:
housing_trim = housing_trim[(housing_trim["housing_fa"] / housing_trim["total_fa"]) > .9]

In [106]:
conditions = [
    (housing_trim['exceeds_2024_limit'] == True),
    (housing_trim['exceeds_2024_limit'] == False) & (housing_trim['exceeds_2030_limit'] == True),
    (housing_trim['exceeds_2024_limit'] == False) & (housing_trim['exceeds_2030_limit'] == False)
]

# Define the choices
choices = ['Exceeds 2024 limit', 'Exceeds 2030 limit', 'Below both limits']

# Create the new column
housing_trim['limit_category'] = np.select(conditions, choices)

In [107]:
housing_trim['apt_size_cat'] = pd.cut(
    housing_trim['avg_apt_size'],
    [0, 750, 1000, 1250, 2000, np.inf],
    labels=["< 750", "750 - 1000", "1000 - 1250", "1250 - 2000", "2000 +"]
)

housing_trim['apt_size_cat2'] = pd.cut(
    housing_trim['avg_apt_size'],
    [0, 650, 1000, 1250, np.inf],
    labels=["< 650", "650 - 1000", "1000 - 1250", "1250+"]
)

housing_trim['sqft_bedroom_cat'] = pd.cut(
    housing_trim['sqft_per_bedroom'],
    [0, 450, 600, 750, 1000, np.inf],
    labels=["< 450", "450 - 600", "600 - 750", "750 - 1000", "1000 +"]
)

In [108]:
energy["Water Use (All Water Sources) (kgal)"].head(1000)

0                   9
1                5965
2                 484
3              4314.9
4              1299.4
            ...      
1048          10390.4
1049    Not Available
1050             6330
1051           6454.2
1052    Not Available
Name: Water Use (All Water Sources) (kgal), Length: 1000, dtype: object

In [109]:
import censusdis.data as ced
import censusdis.geography as cgeo
from censusdis.states import STATE_NY
YEAR = 2020
DATASET = "acs/acs5"
GROUP = "B03002"
CENSUS_VARS = {
    "B17001_001E" : "total_population",
    "B17001_002E" : "pop_in_poverty",
    "B03002_004E" : "pop_black",
    "B03002_012E" : "pop_hispanic",
    "B19013_001E" : "med_hh_inc"
}


In [110]:
nyc_tracts = ced.download(
    DATASET,
    YEAR,
    CENSUS_VARS,
    state=STATE_NY,
    county="*",
    tract = "*",
    with_geometry = True
).rename(CENSUS_VARS, axis = 1)
nyc_tracts["GEOID"] = nyc_tracts["STATE"] + nyc_tracts["COUNTY"] + nyc_tracts["TRACT"]

In [111]:
cols_to_divide = [col for col in nyc_tracts.columns if col.startswith('pop')]
nyc_tracts[cols_to_divide] = nyc_tracts[cols_to_divide].div(nyc_tracts['total_population'], axis=0)

In [112]:
from shapely.geometry import Point

# Create a new geometry column in your DataFrame
housing_trim['geometry'] = housing_trim.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

# Convert your DataFrame to a GeoDataFrame
housing_trim_geo = gpd.GeoDataFrame(housing_trim, geometry='geometry', crs = "EPSG:4269")

In [113]:
housing_trim_geo = housing_trim_geo.overlay(nyc_tracts.to_crs(4269))

In [114]:
housing_trim_geo = housing_trim_geo.query("med_hh_inc > 0")

In [115]:
bins = [0, 30000, 75000, 125000, 175000, np.inf]
labels = ['<30k', '30k-75k', '75k-125k', '125k-175k', '175k+']

# Create a new column with the binned data
housing_trim_geo['income_bin'] = pd.cut(housing_trim_geo['med_hh_inc'], bins=bins, labels=labels)
housing_trim_geo["income_q"] = pd.qcut(housing_trim_geo['med_hh_inc'], q=10, labels = False)

In [116]:
coefs = {'Fuel Oil #2 Use (kBtu)' : 0.00007421,
 'Fuel Oil #4 Use (kBtu)' : 0.00007529,
 'District Steam Use (kBtu)' : 0.00004493,
 'Natural Gas Use (kBtu)' : 0.00005311 ,
 'Electricity Use - Grid Purchase (kWh)' : 0.000288962}

In [117]:
housing["Electricity Use - Grid Purchase (kBtu)"]

1        1642970.2
3        2191128.4
5         703117.6
8        1135718.3
9        1596587.4
           ...    
30462     458361.3
30466     601259.2
30467     618933.4
30477    1169654.1
30478     606776.4
Name: Electricity Use - Grid Purchase (kBtu), Length: 18374, dtype: object

In [118]:
energy_mix = housing[['Property ID', 'Electricity Use - Grid Purchase (kBtu)'] + list(coefs.keys())].apply(pd.to_numeric, errors='coerce').fillna(0)

In [119]:
for col, coef in coefs.items():
    energy_mix[col + "_est"] = energy_mix[col] * coef

In [120]:
# Create 'total' column
cols = [col + "_est" for col in coefs.keys()]

energy_mix['total_emissions'] = energy_mix[cols].sum(axis=1)

# Create proportion columns
for col in cols:
    energy_mix[f'{col}_proportion'] = energy_mix[col] / energy_mix['total_emissions']

In [121]:
energy_mix["fuel_oil_emissions_pct"] = energy_mix["Fuel Oil #2 Use (kBtu)_est_proportion"] + energy_mix["Fuel Oil #4 Use (kBtu)_est_proportion"]
energy_mix["district_steam_emissions_pct"] = energy_mix["District Steam Use (kBtu)_est_proportion"]
energy_mix["natural_gas_emissions_pct"] = energy_mix["Natural Gas Use (kBtu)_est_proportion"]
energy_mix["electricity_emissions_pct"] = energy_mix["Electricity Use - Grid Purchase (kWh)_est_proportion"]

In [122]:
# Create 'total' column
cols = [col for col in energy_mix.columns if col.endswith("(kBtu)")]
cols

['Electricity Use - Grid Purchase (kBtu)',
 'Fuel Oil #2 Use (kBtu)',
 'Fuel Oil #4 Use (kBtu)',
 'District Steam Use (kBtu)',
 'Natural Gas Use (kBtu)']

In [123]:
cols = [col for col in energy_mix.columns if col.endswith("(kBtu)")]

energy_mix['total_energy'] = energy_mix[cols].sum(axis=1)

# Create proportion columns
for col in cols:
    energy_mix[f'{col}energy_proportion'] = energy_mix[col] / energy_mix['total_energy']

In [124]:
energy_mix["fuel_oil_energy_pct"] = energy_mix["Fuel Oil #2 Use (kBtu)energy_proportion"] + energy_mix["Fuel Oil #4 Use (kBtu)energy_proportion"]
energy_mix["district_steam_energy_pct"] = energy_mix["District Steam Use (kBtu)energy_proportion"]
energy_mix["natural_gas_energy_pct"] = energy_mix["Natural Gas Use (kBtu)energy_proportion"]
energy_mix["electricity_energy_pct"] = energy_mix["Electricity Use - Grid Purchase (kBtu)energy_proportion"]

In [125]:

energy_mix['inferred_heating_method'] = np.select(
    [   
        energy_mix['natural_gas_emissions_pct'] > .3,
        energy_mix['fuel_oil_emissions_pct'] > .4, 
        energy_mix['district_steam_emissions_pct'] > .3,
        energy_mix['electricity_emissions_pct'] > .8, 

    ], 
    [
        'Natural Gas',
        'Fuel Oil', 
        'District Steam',
        'Electricty'

    ], 
    default='Unknown'
)

In [126]:
energy_mix

,Property ID,Electricity Use - Grid Purchase (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),District Steam Use (kBtu),Natural Gas Use (kBtu),Electricity Use - Grid Purchase (kWh),Fuel Oil #2 Use (kBtu)_est,Fuel Oil #4 Use (kBtu)_est,District Steam Use (kBtu)_est,...,Electricity Use - Grid Purchase (kBtu)energy_proportion,Fuel Oil #2 Use (kBtu)energy_proportion,Fuel Oil #4 Use (kBtu)energy_proportion,District Steam Use (kBtu)energy_proportion,Natural Gas Use (kBtu)energy_proportion,fuel_oil_energy_pct,district_steam_energy_pct,natural_gas_energy_pct,electricity_energy_pct,inferred_heating_method
1,9793770,1642970.2,0.0,0.0,0.0,1453700.0,481527.0,0.000000,0.0,0.0,...,0.530560,0.000000,0.0,0.0,0.469440,0.000000,0.0,0.469440,0.530560,Natural Gas
3,13511507,2191128.4,0.0,0.0,0.0,5122899.9,642182.9,0.000000,0.0,0.0,...,0.299579,0.000000,0.0,0.0,0.700421,0.000000,0.0,0.700421,0.299579,Natural Gas
5,14377690,703117.6,0.0,0.0,0.0,5199600.0,206072.0,0.000000,0.0,0.0,...,0.119118,0.000000,0.0,0.0,0.880882,0.000000,0.0,0.880882,0.119118,Natural Gas
8,15176247,1135718.3,414000.0,0.0,0.0,8852599.8,332860.0,30.722940,0.0,0.0,...,0.109179,0.039799,0.0,0.0,0.851022,0.039799,0.0,0.851022,0.109179,Natural Gas
9,15176327,1596587.4,588087.1,0.0,0.0,8681600.0,467932.9,43.641944,0.0,0.0,...,0.146931,0.054120,0.0,0.0,0.798949,0.054120,0.0,0.798949,0.146931,Natural Gas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30462,6699459,458361.3,0.0,0.0,0.0,3067877.2,134338.0,0.000000,0.0,0.0,...,0.129986,0.000000,0.0,0.0,0.870014,0.000000,0.0,0.870014,0.129986,Natural Gas
30466,6681872,601259.2,2347656.0,0.0,0.0,164300.0,176219.0,174.219552,0.0,0.0,...,0.193131,0.754094,0.0,0.0,0.052775,0.754094,0.0,0.052775,0.193131,Fuel Oil
30467,6682201,618933.4,2398577.8,0.0,0.0,153400.0,181399.0,177.998459,0.0,0.0,...,0.195191,0.756432,0.0,0.0,0.048377,0.756432,0.0,0.048377,0.195191,Fuel Oil
30477,6875712,1169654.1,0.0,0.0,0.0,6417199.4,342806.0,0.000000,0.0,0.0,...,0.154169,0.000000,0.0,0.0,0.845831,0.000000,0.0,0.845831,0.154169,Natural Gas


In [127]:
energy_mix_join = energy_mix[["Property ID", "total_energy", "total_emissions", "district_steam_emissions_pct", "natural_gas_emissions_pct",
                               "electricity_emissions_pct", "fuel_oil_emissions_pct", "fuel_oil_energy_pct", "district_steam_energy_pct", "natural_gas_energy_pct", "electricity_energy_pct", "inferred_heating_method"]]
housing_trim_geo = housing_trim_geo.merge(energy_mix_join)

In [128]:
# Convert categorical columns to string
for col in housing_trim_geo.select_dtypes(include='category').columns:
    housing_trim_geo[col] = housing_trim_geo[col].astype(str)

housing_trim_geo.columns = housing_trim_geo.columns.str.lower()

In [129]:
pluto = pd.read_csv("https://data.cityofnewyork.us/api/views/64uk-42ks/rows.csv?date=20231212&accessType=DOWNLOAD")
bor_map = {"BK" : "3",
           "BX" : "2",
           "MN" : "1",
           "SI" : "5",
           "QN" : "4"}
pluto["borough"] = pluto["borough"].map(bor_map)

/var/folders/ct/9lwc0fbn26v_v6pz1pfj1dxc0000gq/T/ipykernel_10454/2150334008.py:1: DtypeWarning: Columns (17,18,20,21,22,24,25,52,61,62,79,97) have mixed types. Specify dtype option on import or set low_memory=False.


In [130]:
pluto['block'] = pluto['block'].apply(lambda x: str(x).zfill(5))
pluto['lot'] = pluto['lot'].apply(lambda x: str(x).zfill(4))
pluto["bbl"] = pluto["borough"] + pluto["block"] + pluto["lot"]

In [131]:
housing_trim_geo = housing_trim_geo.merge(pluto[["bbl", "yearalter1", "yearalter2", "assesstot"]])
housing_trim_geo["ever_altered"] = housing_trim_geo["yearalter1"] != 0
housing_trim_geo["altered_twice_or_more"] = housing_trim_geo["yearalter2"] != 0

In [132]:
housing_trim_geo["value_per_apt"] = housing_trim_geo["assesstot"] / housing_trim_geo["total_units"] / .06
housing_trim_geo["value_per_sqft"] = housing_trim_geo["assesstot"] / housing_trim_geo["total_fa"] / .06

In [133]:
housing_trim_geo.to_file("../data/ghg_processed.geojson")

In [134]:
housing_trim_geo.query("total_co2.isna()")

,bbl,name,property id,total_units,eui,bedrooms,housing_fa,total_fa,co2_intensity,total_co2,...,natural_gas_energy_pct,electricity_energy_pct,inferred_heating_method,yearalter1,yearalter2,assesstot,ever_altered,altered_twice_or_more,value_per_apt,value_per_sqft
3,2042520017,A02760 - Parkway Owners Inc,15289404,52.0,NaN,84.0,44329.0,44329.0,NaN,NaN,...,0.000000,0.000000,Fuel Oil,0.0,0.0,1729350.0,False,False,5.542788e+05,650.195132
488,4012220009,AAR: 65-15 38 Avenue,2801835,145.0,NaN,145.0,139100.0,139100.0,NaN,NaN,...,1.000000,0.000000,Natural Gas,0.0,0.0,3060000.0,False,False,3.517241e+05,366.642703
582,1021360235,FORT WASHINGTON AVENUE REHAB,2831038,226.0,NaN,249.0,293550.0,293550.0,NaN,NaN,...,0.000000,1.000000,Electricty,0.0,0.0,10984050.0,False,False,8.100332e+05,623.633112
709,1021340022,Ymy Acquisitions : 780 RIVERSIDE DRIVE,21715318,71.0,NaN,71.0,119049.0,119049.0,NaN,NaN,...,NaN,NaN,Unknown,1985.0,0.0,3279150.0,True,False,7.697535e+05,459.075675
765,1020260041,208 W 141 St,14764393,24.0,NaN,52.0,25410.0,25410.0,NaN,NaN,...,0.131943,0.868057,Electricty,1986.0,1999.0,333900.0,True,True,2.318750e+05,219.008264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12632,1000417501,A02600 - 50 Pine Street Condominium,10819501,20.0,NaN,43.0,45352.0,45352.0,NaN,NaN,...,0.000000,0.000000,Fuel Oil,0.0,0.0,6217199.0,False,False,5.180999e+06,2284.794129
12819,1004300010,FIRST HOUSES,2831034,126.0,NaN,135.0,107538.0,107538.0,NaN,NaN,...,NaN,NaN,Unknown,0.0,0.0,11880000.0,False,False,1.571429e+06,1841.209619
13849,2029440001,Murphy Building 2,6818175,132.0,NaN,305.0,146250.0,146250.0,NaN,NaN,...,NaN,NaN,Unknown,2018.0,2018.0,10171800.0,True,True,1.284318e+06,1159.179487
14002,1016780001,325 E 106th St,11346673,117.0,NaN,176.0,124000.0,124000.0,NaN,NaN,...,0.000000,1.000000,Electricty,0.0,0.0,31421700.0,False,False,4.476026e+06,4223.346774
